In [59]:
%load_ext autoreload
%autoreload 2

import docker
import logging
from jtldockerctl.dctl import *
import hashlib
from names_generator import generate_name
from jtlutil.config import get_config

config = get_config('./config.env')

logging.basicConfig(level=logging.ERROR)
logger.setLevel(logging.DEBUG)

client = docker.DockerClient(base_url=config.SSH_URI )

ensure_network_exists(client, 'x11', is_external=False, network_type="bridge")
ensure_network_exists(client, 'jtlctl', is_external=False, network_type="bridge")
ensure_network_exists(client, 'caddy', is_external=True, network_type="bridge")


DEBUG:jtldockerctl.dctl:Network 'x11' already exists.
DEBUG:jtldockerctl.dctl:Network 'jtlctl' already exists.
DEBUG:jtldockerctl.dctl:Network 'caddy' already exists.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<Network: 341c37b26522>

In [ ]:
def make_servers(usernames):
    servers = []
    for username in usernames:
        nvc, pa = create_cs_pair(client, config, config.IMAGES_PYTHONCS, username)
        yield (nvc, pa)
    return servers

print('----')
for nvc, pa in make_servers(['jed','osvaldo', 'eric']):
    print(nvc.id, pa.labels['caddy'])


----
621e6611a5eb584f6cac387f4ad7021be0d4afca90c15432f34e74fd8177f6b8 jed.do.jointheleague.org
af73e62f054b2d8957dddaaf5e7ec6a929469fd8194234bbe04aab26bf9ee731 osvaldo.do.jointheleague.org
e646249ba6c6cc72b9a6b2e9fbb6993b0415292fcbf877136a8ef873f511ab5c eric.do.jointheleague.org


In [57]:
list(make_servers(['jed']))

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])

DEBUG:jtldockerctl.dctl:Found 1 containers
DEBUG:jtldockerctl.dctl:Starting container jed-novnc
DEBUG:jtldockerctl.dctl:Found 1 containers
DEBUG:jtldockerctl.dctl:Starting container jed


9e08d050c03999033981b000a0484000d51c09f1c5d5e5466940ef602e018ea7 running jed.do.jointheleague.org
1d2919631353a9f52dd719c52ef4bf1b6b36f64026d38c01302ac19aee3acde5 running jed-novnc.do.jointheleague.org


In [ ]:

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])
    if c.status == 'running':
        c.stop()
        #c.remove()

In [19]:
pa.labels['caddy']

'eric.do.jointheleague.org'